In [4]:
'''
1. from ?ㅼ뿉 ?뚯씠釉?a

2. insert overwrite OR create table. from ?욎뿉.. ?뚯씠釉?b

  a->b

3. join ?뚯씠釉?c, d
  a->b ?곹깭?먯꽌 a->b, c->b, d->b

'''

'\n1. from \xeb\x92\xa4\xec\x97\x90 \xed\x85\x8c\xec\x9d\xb4\xeb\xb8\x94 a\n\n2. insert overwrite OR create table. from \xec\x95\x9e\xec\x97\x90.. \xed\x85\x8c\xec\x9d\xb4\xeb\xb8\x94 b\n\n  a->b\n\n3. join \xed\x85\x8c\xec\x9d\xb4\xeb\xb8\x94 c, d\n  a->b \xec\x83\x81\xed\x83\x9c\xec\x97\x90\xec\x84\x9c a->b, c->b, d->b\n\n'

In [5]:

import os
import sys
import re

def ls(dir, hidden=False, relative=True):
    nodes = []
    for nm in os.listdir(dir):
        if not hidden and nm.startswith('.'):
            continue
        if not relative:
            nm = os.path.join(dir, nm)
        nodes.append(nm)
    nodes.sort()
    return nodes

#ext = file extension
def find(root, ext='', files=True, dirs=False, hidden=False, relative=True, topdown=True):
    root = os.path.join(root, '')  # add slash if not there
    for parent, ldirs, lfiles in os.walk(root, topdown=topdown):
        if relative:
            parent = parent[len(root):]
        if dirs and parent:
            yield os.path.join(parent, '')
        if not hidden:
            lfiles   = [nm for nm in lfiles if not nm.startswith('.')]
            ldirs[:] = [nm for nm in ldirs  if not nm.startswith('.')]  # in place
        if files:
            lfiles.sort()
            for nm in lfiles:
                if len(ext) == 0 or (len(ext)>0 and nm.find('.'+ext, len(nm)-1-len(ext)) > 0):
                    nm = os.path.join(parent, nm)
                    yield nm
                else:
                    continue

In [6]:
def clean_line(line):
    line = line.strip()
    line = line.replace('\t', ' ')
    line = line.replace('   ', ' ')
    line = line.replace('  ', ' ')
    line = line.replace(' ,', ',')
    line = line.replace(' ;', ';')
    line = line.replace(' )', ')')
    line = line.replace('( ', '(')
    return line

In [30]:
#find table relationships
def process_sql(query):
    print ">> def process_sql"
    dic = {}
    query = query.lower()
    
    #find from_table. cannot find multiple from_tables
    from_pos = query.find('from ')
    if from_pos < 0:
        print "NO 'from' found!"
        return {}
    
    from_line = query[from_pos+5:]
    searchObj = re.search(r'[a-z0-9_\.]*', from_line, re.M|re.I)

    from_table = ''
    if searchObj:
        from_table = searchObj.group()

    print '['+from_table+']'

    #find to_table
    line = query[:from_pos]
    table_pos = line.find('table ')
    if table_pos < 0:
        print "NO dest table found!!"
        return {}
    table_pos = table_pos + len('table ')
    
    exists_pos = line.find('exists ')
    if exists_pos > 0:
        table_pos = exists_pos + len('exists ')
    
    searchObj = re.search(r'[a-z0-9_\.]*', line[table_pos:], re.M|re.I)

    to_table = ''
    if searchObj:
        to_table = searchObj.group()

    print '['+to_table+']'
    
    #find join_tables
    join_pos = from_line.find(' join')
    while join_pos > 0:
        join_pos = join_pos + len(' join')
        from_pos = -1
        if from_line.find('(', join_pos, join_pos+2) > 0:
            from_pos = from_line.find('from ', join_pos+2) + 5
            from_line = from_line[from_pos:]
        else:
            from_line = from_line[join_pos:]
            
        from_line = clean_line(from_line)
            
        searchObj = re.search(r'[a-z0-9_\.]*', from_line, re.M|re.I)

        from_table = ''
        if searchObj:
            from_table = searchObj.group()

        print 'JOIN['+from_table+']'
        
        from_line = from_line[5+len(from_table):]
        join_pos = from_line.find(' join')
    
    return dic

In [33]:
#returns dic of table flows, a_table -> b_table, b_table -> c_table. or empty dic
def process_file(filepath):
    dic = {}
    
    print ("File : [" + filepath + "]")
    f = open(filepath, 'r')
    if not f:
        return dic

    lines = f.readlines()
    f.close()

    sqls = []
    sql = ''
    for line in lines:
        line = clean_line(line)
        #skip comment lines
        comment_pos = line.find('--')
        if comment_pos < 0: #not a comment
            if len(sql) > 0 and sql[-1] != ' ':
                line = ' ' + line
            sql = sql + line
            #print sql
            #print '~~~~~~~~~~~~~~~~~~'
            if line and line[-1] == ';':
                sqls.append(clean_line(sql))
                sql = ''

    num = 1
    for s in sqls:
        print num
        num = num + 1
        process_sql(s.lower())
        #print(s.lower())
    
    return dic

In [34]:
root = 'D:\work'
for hqlfile in find(root, ext='sql',dirs=False, hidden=True):
    hqlpath = root+'\\'+hqlfile
    process_file(hqlpath)
    break

File : [D:\work\a\test.sql]
1
>> def process_sql
NO 'from' found!
2
>> def process_sql
[pap_701_ki.tm701_qm_device_primary_dana]
[tm711_qm_btr_dou_wana_dvc_id]
3
>> def process_sql
[pap_703.gear_device_new]
[tm711_qm_btr_dou_wana_dvc_id]
4
>> def process_sql
NO 'from' found!
5
>> def process_sql
[default.tm711_qm_btr_dou_wana_dvc_id]
[tm711_qm_btr_dou_wana]
JOIN[pap_701_mart.tm701_dvc_modl_id]
JOIN[pap_701_mart.tm701_dvc_grp_id]
JOIN[pap_701_mart.tm701_tco_cnty_rgn]


In [29]:
line = ' pap_701_mart.tm701_tco_cnty_rgn as t_mnc_mcc on (cast(t_dou.mcc_cd as int) = cast(t_mnc_mcc.mcc_cd as int) and cast(t_dou.mnc_cd as int) = cast(t_mnc_mcc.mnc_cd as int)) group by t_mnc_mcc.cnty_nm, t_mnc_mcc.rgn_cd, t_mnc_mcc.mnc_nm, t_modl.dvc_gp_nm, t_dou.model_id, t_dou.fw_ver, floor(t_dou.dou);'
searchObj = re.search(r'[ a-z0-9_\.]*', line, re.M|re.I)

from_table = ''
if searchObj:
    from_table = searchObj.group()
    print 'JOIN['+from_table+']'


JOIN[ pap_701_mart.tm701_tco_cnty_rgn as t_mnc_mcc on ]


In [11]:
hqlpath

'D:\\work\\a\\test.sql'